In [1]:
# !pip install tqdm
# !pip install time

# %pip install --force-reinstall -v "ipywidgets == 7.7.2"
# %pip install --force-reinstall -v "jupyterlab_widgets == 1.1.1"

In [2]:
# ALL IMPORTS FOR CODE 

import os
import sys
import numpy as np
import pandas as pd
import time
import json
import requests
import shutil
import PIL.Image
from IPython.display import display
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from pandas import json_normalize
from IPython.display import Image, HTML
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
# ALL GLOBAL FUNCITONS

def path_to_image_html(path):
    return '<img width="500" src="'+ path + '"/>'


def download_image(url):
    filepath = os.path.join("img", os.path.basename(url))
    if not os.path.exists("img"):
        # Create a new directory because it does not exist
        os.makedirs("img")
        # print("The new directory is created!")
    headers = {"User-Agent": "Mozilla/5.0"}

    
    if os.path.isfile(filepath) :
        # print(f"File {os.path.basename(url)} already exists")
        return filepath

    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open(filepath, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    # print(f"File {os.path.basename(url)} downloaded")
    return filepath

In [6]:

endpoint_url = "https://query.wikidata.org/sparql"
imgmax = 1000

# Get cities
query = """SELECT DISTINCT ?planeLabel ?entry ?image {
  ?plane wdt:P31 wd:Q15056993;
               wdt:P729 ?entry;
               wdt:P729 ?retirement;
               wdt:P18 ?image.
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
} LIMIT 1000"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)
res = results["results"]["bindings"]
i = 0
for result in tqdm(res):
    i+=1
    # print(f"{i} / {len(res)} |  ", end='')
    filename, file_extension = os.path.splitext(os.path.basename(result["image"]["value"]))
    if file_extension not in [".png", ".jpg"] :
        # print(f"File {filename} SKIPPED")
        continue
    path = download_image(result["image"]["value"])
    img = PIL.Image.open(path)
    exif_data = img._getexif()
    array.append(
        (
            result["planeLabel"]["value"],
            result["entry"]["value"],
            result["image"]["value"],
            exif_data
        )
    )


dataframe = pd.DataFrame(array, columns=["planeLabel", "entry", "image", "data"])
dataframe = dataframe.astype(
    dtype={"planeLabel": "<U200", "entry" : "<U200", "image": "<U200", "data": "<U200"}
)
srt = dataframe.sort_values("entry")

pd.set_option('display.max_colwidth', 100)

HTML(srt.to_html(escape=False ,formatters=dict(image=path_to_image_html)))

100%|██████████| 551/551 [00:00<00:00, 4167.69it/s]


,planeLabel,entry,image,data
300,Blériot XI,1909-01-01T00:00:00Z,,"{296: 2, 34665: 164, 305: 'Adobe Photoshop CS Windows', 274: 1, 306: '2005:09:05 20:43:23', 282: 300.0, 283: 300.0, 40961: 1, 40962: 1280, 40963: 582}"
347,AEG B,1914-01-01T00:00:00Z,,None
427,AEG B,1914-01-01T00:00:00Z,,None
22,LVG C.II,1915-01-01T00:00:00Z,,None
333,Sopwith 1½ Strutter,1916-04-01T00:00:00Z,,None
393,Sopwith Camel,1917-06-01T00:00:00Z,,"{4097: 1600, 4098: 1200, 34665: 144, 271: 'Canon', 272: 'Canon PowerShot G5', 306: '2005:01:29 13:44:59', 531: 1, 36864: b'0220', 37121: b'\x01\x02\x03\x00', 37122: 5.0, 36867: '2005:01:29 13:44:59', 36868: '2005:01:29 13:44:59', 37377: 5.65625, 37378: 2.0, 37380: -0.6666666666666666, 37381: 2.0, 37383: 2, 37385: 24, 37386: 7.1875, 37510: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 40961: 1, 40962: 1600, 41486: 5653.7102473498235, 40963: 1200, 41487: 5660.377358490566, 41488: 2, 41495: 2, 41728: b'\x03', 33434: 0.02, 33437: 2.0, 41985: 0, 41986: 1, 40960: b'0100', 41987: 0, 41988: 1.0, 37500: b'\x0f\x00\x01\x00\x03\x00.\x00\x00\x00h\x04\x00\x00\x02\x00\x03\x00\x04\x00\x00\x00\xc4\x04\x00\x00\x03\x00\x03\x00\x04\x00\x00\x00\xcc\x04\x00\x00\x04\x00\x03\x00""\x00\x00\x00\xd4\x04\x00\x00\x00\x00\x03\x00\x06\x00\x00\x00\x18\x05\x00\x00\x00\x00\x03\x00\x04\x00\x00\x00$\x05\x00\x00\x12\x00\x03\x00\x1c\x00\x00\x00,\x05\x00\x00\x13\x00\x03\x00\x04\x00\x00\x00d\x05\x00\x00\x06\x00\x02\x00 \x00\x00\x00l\x05\x00\x00\x07\x00\x02\x00\x18\x00\x00\x00\x8c\x05\x00\x00\x08\x00\x04\x00\x01\x00\x00\x00WB\x0f\x00\t\x00\x02\x00 \x00\x00\x00\xa4\x05\x00\x00\x10\x00\x04\x00\x01\x00\x00\x00\x00\x00)\x01\x00\x00\x03\x00\x05\x00\x00\x00\xc4\x05\x00\x00\r\x00\x03\x00""\x00\x00\x00\xce\x05\x00\x00\x00\x00\x00\x00\\\x00\x02\x00\x00\x00\x05\x00\x05\x00\x00\x00\x00\x00\x04\x00\x00\x00\x01\x00\x05\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x05\x00\x01\x00\x05 \x01\x00\xff\xff\xff\xff\x9a\x03\xe6\x00 \x00C\x00\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\xff\xff\x00\x00 \n \n\x00\x00\x00\x00\x00\x00\x00\x00\xff\x7f\x00\x00\x00\x00\x00\x00\x02\x00\xe6\x00""\x01\xd9\x00\x00\x04\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x80\x00a\x00@\x00\xb5\x00\xec\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\xf9\x01\x00\x00C\x00\xb3\x00\x00\x00\x00\x00\x00\x00\xfa\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t\x00\x01\x00@\x06\xb0\x04 \n\xd7\x00\xd3\x01\'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00IMG:PowerShot G5 JPEG\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Firmware Version 1.00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x00\x02\x00\x02\x00\x80\x02\xe0\